In [91]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy


In [92]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [93]:
# some print for me to understand
sets = make_set_covering_problem(3,6, 0.3)
print(sets)
print(sets.shape[0])
print(sets.getrow(0).toarray().ravel())

  (0, 2)	True
  (1, 0)	True
  (1, 2)	True
  (2, 0)	True
  (2, 2)	True
  (3, 2)	True
  (4, 1)	True
  (4, 2)	True
  (5, 1)	True
6
[False False  True]


### Basically I set an initial array of all False and I tried using all the sets in order to find the best improvement at each step. In order to find the best combination I just counted the number of covered elements. I did this to make a comparison with the Hill Climber solution implemented later. 

In [94]:
NUM_POINTS = 1000 # try [100, 1_000, 5_000]
NUM_SETS = NUM_POINTS
DENSITY = .3

In [95]:
def set_cover(sets):
    covered = np.zeros(NUM_POINTS, dtype=bool)
    chosen_sets = [] # array that contains the indexes of the chosen sets

    while not all(covered):
        max_covered_points = 0
        best_set = 0
        for i in range(sets.shape[0]):
            set = sets.getrow(i).toarray().ravel() # row represented in the following form: [False True True False ...]
            covered_points = np.logical_or(set, covered).sum()
            if covered_points > max_covered_points:
                max_covered_points = covered_points
                best_set = i

        chosen_sets.append(best_set)
        covered.__ior__(sets.getrow(best_set).toarray().ravel()) # update with OR operator in-place

    return chosen_sets

In [96]:
sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
chosen_sets = set_cover(sets)
print("Chosen sets:", chosen_sets)

Chosen sets: [714, 404, 991, 572, 736, 951, 153, 212, 113, 6]


## Random-Mutation Hill Climber implementation

Here I implemented the Random-Mutation Hill Climber algorithm adding the possibility of doing bigger steps after that a certain amount of iterations without improvement is done. Furthermore I added a constraint on the number of max iterations without improvement.

In [97]:
NUM_POINTS = 1000 # try [100, 1_000, 5_000]
NUM_SETS = NUM_POINTS
DENSITY = .3

In [98]:
def random_tweak(sets, indexes, counter_untill_longer_jump):
    try_longer_jump = 1000
    step_longer_jump = int(NUM_POINTS/50)
    new_state = np.zeros(NUM_POINTS, dtype=bool)
    new_indexes = copy(indexes)
    index = randint(0, NUM_SETS - 1)
    if counter_untill_longer_jump > try_longer_jump:
        for i in range(step_longer_jump): # try a bigger change
            index = randint(0, NUM_SETS - 1)
            new_indexes[index] = not new_indexes[index]
    new_indexes[index] = not new_indexes[index]
    for i in range(len(new_indexes)):
        if new_indexes[i] == 1:
            new_state = new_state.__ior__(sets.getrow(i).toarray().ravel())
    return new_state, new_indexes, counter_untill_longer_jump

def fitness(state, indexes):
    cost = sum(indexes)
    goal = np.sum(state)
    return goal, -cost


In [99]:
def rm_hill_climber(sets, n_iterations):
    state = np.zeros(NUM_POINTS, dtype=bool)
    indexes = np.zeros(NUM_SETS)
    counter_until_longer_jump = 0

    best_fitness = fitness(state, indexes)
    for step in range(10_000):
        new_state, new_indexes, counter_until_longer_jump = random_tweak(sets, indexes, counter_until_longer_jump)
        fit_values = fitness(new_state, new_indexes)
        n_iterations += 1
        if fit_values >= best_fitness:
            best_fitness = fit_values
            state = new_state
            indexes = new_indexes
            #print(fitness(state, indexes))
            counter_until_longer_jump = 0
        else:
            counter_until_longer_jump += 1
            if counter_until_longer_jump >= 1100: #break if you reach this number of iterations without improvement
                break
    
    return indexes, n_iterations

In [100]:
sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
n_iterations = 0
indexes_chosen_sets, n_iterations = rm_hill_climber(sets, n_iterations)
print(f"Chosen sets in the problem with NumPoints=Numsets={NUM_POINTS} and density={DENSITY}:")
for i in range(len(indexes_chosen_sets)):
    if indexes_chosen_sets[i] == 1:
        print(i, end = " ")


Chosen sets in the problem with NumPoints=Numsets=1000 and density=0.3:
66 139 194 210 294 453 458 486 543 707 746 900 912 917 924 

In [101]:
for DENSITY in [.3, .7]:
    for NUM_POINTS in [100, 1000, 5000]:
        NUM_SETS = NUM_POINTS
        n_iterations = 0
        sets = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
        indexes_chosen_sets, n_iterations = rm_hill_climber(sets, n_iterations)
        print(f"NumPoints=Numsets={NUM_POINTS} density={DENSITY} -> best fit={int(sum(indexes_chosen_sets))} - fitness calls={n_iterations}")


NumPoints=Numsets=100 density=0.3 -> best fit=8 - fitness calls=1284
NumPoints=Numsets=1000 density=0.3 -> best fit=15 - fitness calls=1923
NumPoints=Numsets=5000 density=0.3 -> best fit=22 - fitness calls=1124
NumPoints=Numsets=100 density=0.7 -> best fit=3 - fitness calls=1329
NumPoints=Numsets=1000 density=0.7 -> best fit=6 - fitness calls=1106
NumPoints=Numsets=5000 density=0.7 -> best fit=7 - fitness calls=1193
